<a href="https://colab.research.google.com/github/vkaramanis/Coursera_Capstone/blob/main/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Let's assume that there is a small travel agency that wants to personalize its recommendations depending on the clients' preferences. This will enable the agency to understand how similar are different places and provide targeted suggestions to clients that have particular preferences.

Such a move will be beneficial for the clients too as they will have the opportunity to visit places that they most probably like rather than randomly visiting a place.

To do that we will use forthsquare's top picks for the 100 most popular destinations according to wikipedia [List_of_cities_by_international_visitors](https://en.wikipedia.org/wiki/List_of_cities_by_international_visitors).

We will classify the cities based on the similarities they have on these top picks. Based on this classification we will find which are the key characteristics for each group of cities. This will enable us to provide targeted suggestions to future clients.
